In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "."]).decode("utf8"))

# Any results you write to the current directory are saved as output.

Boston Univ
Involved
Machine Learning
Nitish-Gaddam-CIFAR-10.ipynb
Print
Project commands used.docx
Project_767.ipynb
Project_767_1.ipynb
Project_design_nitish.docx
Random
Resume
Sublime Text 3
TensorFlow
Untitled.ipynb
Untitled1.ipynb
buddha.jpg
download.py
dumbfuck
experiment.py
jobs.xlsx
newdata
nish.py
prexel
python3.docx
~$oject commands used.docx
~$ython3.docx



In [2]:
import tensorflow as tf
import numpy as np
from IPython.display import YouTubeVideo

tf.reset_default_graph()

video_lvl_record = "/Users/gaddamnitish/features/train0-.tfrecord"
frame_lvl_record = "/Users/gaddamnitish/features/train0-.tfrecord"

# now, let's read the frame-level data
# due to execution time, we're only going to read the first video

feat_rgb = []
feat_audio = []
targets = []

num_labels = 4716

def createTargetVec(labels):
    out = np.zeros((1, num_labels))
    for label in labels:
        out[0,label] = 1
    return out

samples = 15
base = np.empty((samples, 100, 128))
tars = np.empty((samples, num_labels))
 

In [3]:
# there are 4716 different labels. therefore we need to predict a vector
# with probabilities. the 5 labels with the highest probability will then be selected 
# for submission.
k = 0
sess = tf.InteractiveSession()
for example in tf.python_io.tf_record_iterator(frame_lvl_record):        
    tf_seq_example = tf.train.SequenceExample.FromString(example)
    labels = tf_seq_example.context.feature['labels'].int64_list.value
    n_frames = len(tf_seq_example.feature_lists.feature_list['audio'].feature)
    rgb_frame = []
    audio_frame = []
    frame = np.zeros((100, 128))
    for i in range(100):
        
#        f = rgb_frame.append(tf.cast(tf.decode_raw(
#                tf_seq_example.feature_lists.feature_list['rgb'].feature[i].bytes_list.value[0],tf.uint8)
#                   ,tf.float32).eval())
        
        
        f = tf.cast(tf.decode_raw(
                tf_seq_example.feature_lists.feature_list['audio'].feature[i].bytes_list.value[0],tf.uint8)
                   ,tf.float32).eval()
    frame[i] = f
    base[k] = frame
    tars[k] = createTargetVec(labels[:])
    k += 1
    progress = (k / samples) * 100
    if int(progress) % 10 == 0:
        print("Progress", progress, "%")
    if k >= samples:
        break
sess.close()

IndexError: list index out of range

In [4]:
    
feat_audio_ = np.asarray(feat_audio)
targets_ = np.asarray(tars)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(base, tars, test_size=0.2, random_state=42)
    
print(len(X_train), 'train sequences')
print(len(X_test), 'test sequences')
print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)
    

12 train sequences
3 test sequences
X_train shape: (12, 100, 128)
X_test shape: (3, 100, 128)
y_train shape: (12, 4716)
y_test shape: (3, 4716)


In [5]:

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

model = Sequential()
model.add(LSTM(128, input_shape=X_train.shape[1:], return_sequences=True))
model.add(LSTM(128, return_sequences=True))
model.add(LSTM(128))
model.add(Dense(128, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(num_labels, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=60, batch_size=64)


ModuleNotFoundError: No module named 'keras'